In [1]:
#version: 3.0 2021/12/19

import os
import sys
import re
sys.path.append("../")

from utils.io import read_csv, load_json, write_to

In [2]:
def E_trans_to_C(string):
    E_pun = u'~:;「」妳著,.!?[]()<>"\''
    C_pun = u'～：；“”你着，。！？【】（）《》“‘'
    table= {ord(f):ord(t) for f,t in zip(E_pun,C_pun)}
    return string.translate(table)

def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248

        rstring += chr(inside_code)

    

    return E_trans_to_C(rstring)

def get_sighan_from_json():

    all_data = {
        "train":None,
        "dev":None,
        "test":None,
        "test14":None,
        "test15":None,
    }
    data_dir = "../data/rawdata/sighan/csc/"

    train_file1 = os.path.join(data_dir, "train_dev.json")
    train_file2 = os.path.join(data_dir, "train131415.json") 
    test14_file = os.path.join(data_dir, "test14.json")
    test15_file = os.path.join(data_dir, "test15.json")

    all_data["train"] = load_json(train_file1)
    all_data["train"].extend(load_json(train_file2))

    all_data["train"] = all_data["train"]

    all_data["valid14"] = load_json(test14_file)
    all_data["valid"] = load_json(test15_file)
    #all_data["test"].extend(load_json(test15_file))

    return all_data

def json2list(data, need_preprocess):
    source, target, ids = [], [], []

    for element in data:

        if need_preprocess:
            source.append(element["original_text"])
            target.append(element["correct_text"])
            ids.apoend(element["wrong_ids"])
        else:
            source.append(strQ2B((element["original_text"])))
            target.append(strQ2B((element["correct_text"])))
            ids.append(element["wrong_ids"])

    return source, target, ids

data = get_sighan_from_json()

source, target, ids = json2list(data["valid"], False)
#source, target, ids = json2list(data["valid"], False)


In [24]:
source = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/data/rawdata/sighan/raw/valid14.src")
target = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/data/rawdata/sighan/raw/valid14.tgt") 

In [25]:
source = [re.sub("\n", "", i) for i in source]
target = [re.sub("\n", "", i) for i in target] 

In [3]:

lattice = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/data/rawdata/sighan/lattice/test.src")

lattice = [ {key:0 for key in lattice[i].split(',')[0].split()} for i in range(1, len(lattice), 2)]

#raw_bert_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_base_std.epoch10.bs128/generated_predictions.txt")
raw_bert_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SE_tmp_back/tmp/sighan/bert_MaskedLM_base_raw.epoch10.bs128/generated_predictions.txt")

#proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bshert_Flat_std_char.epoch10.bs16/generated_predictions.txt")
#proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_v2_std.epoch10.bs32/generated_predictions.txt")
#proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_v2_std_mask.epoch20.bs32/generated_predictions.txt")
#proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_v2_std_mask_dropout02_test.epoch10.bs32/generated_predictions.txt")
#proto_preds = read_csv(_path("bert_MaskedLM_v2_std_mask_expand_d3.epoch10.bs32"))
#proto_preds = read_csv(_path("bert_MaskedLM_v2_std_mask_expand.epoch10.bs32"))

#proto_preds = [ proto_preds[i] for i in range(0, len(proto_preds), 2) ]

#proto_preds = read_csv("./no_mask_preds.txt")
#proto_preds = read_csv("./no_mask_preds_super.txt")
#proto_preds = read_csv("./no_mask_preds_overfit.txt")
#proto_preds = read_csv("./no_mask_bert_preds_overfit.txt")


for i in range(len(source[:1100])):
    raw_bert_preds[i] = E_trans_to_C("".join(raw_bert_preds[i].split())[:len(source[i])])
    #proto_preds[i] = "".join(proto_preds[i].split())[:len(source[i])] 

In [26]:
#lets get which bert wrong

data = get_sighan_from_json()

train_source, train_target, train_ids = json2list(data["train"], False)

valid14_source, valid14_target, valid14_ids = json2list(data["valid14"], False)

valid_source, valid_target, valid_ids = json2list(data["valid"], False)

def get_magic_word(data, ids):
    """
            法新社记者报导,法国驻巴蓦斯坦大使杰拉德,以及巴国官员在巴基斯坦西北部的托克哈姆边界关卡迎接十月九日与两名巴基斯坦同业一起遭塔利班逮捕的裴哈。
    data : "法新社记者报导,法国驻巴基斯坦大使杰拉德,以及巴国官员在巴基斯坦西北部的托克哈姆边界关卡迎接十月九日与两名巴基斯坦同业一起遭塔利班逮捕的裴哈。"
    
    ids: [12]
    """
    #print(data, ids)
    if not ids:
        return ""

    spans = [ i for i in jieba.cut(data) ]
    
    result = []

    pointer_ids = 0
    
    pointer = 0
    #print(spans)
    for span in spans:

        pointer += len(span)
        
        tmp = ""
        if pointer > ids[pointer_ids]:
            span = re.sub("\W*", "", span)
            if span and not span.isdigit():
                tmp = span
            
            if tmp:
                result.append(tmp)

            pointer_ids += 1
            if pointer_ids >= len(ids):
                break
    
    return result


In [28]:
magic_dict = []

from tqdm import tqdm
import jieba
for data, id in tqdm(zip(valid_target, valid_ids)):
    magic_dict.append(get_magic_word(data, id))

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.701 seconds.
Prefix dict has been built successfully.
1100it [00:00, 1364.57it/s]


In [36]:
wrongs = 0
for i in magic_dict:
    if i:
        wrongs += 1
print(wrongs)

542


In [75]:
import jieba


cut_target = [ list(jieba.cut(i)) for i in valid_target ]

cut_bert = [ list(jieba.cut(j)) for j in raw_bert_preds]


In [89]:
word_wrongs = 0

word_wrongs_hit = 0

for i in range(len(cut_target)):
    
    if not valid_target[i] == raw_bert_preds[i]:#bert fail
        
        for j in range(len(cut_target[i])):
            if len(cut_target[i]) != len(cut_target[i]):
                print(cut_target[i], cut_target[i])

            if cut_target[i][j] != cut_bert[i][j] and len(cut_target[i][i]) > 1:
                #word need correcction but not
                word_wrongs += 1
                if cut_target[i][j] in magic_dict[i]:
                    word_wrongs_hit += 1

print(word_wrongs, word_wrongs_hit)


IndexError: list index out of range

In [69]:

cover_count = 0

for i in range(len(valid_target)):
    if not magic_dict[i]:
        continue
    #print(magic_dict[i], lattice[i])
    for j in magic_dict[i]:
        if len(j) > 1 and j not in lattice[i]:
            break
    else:
        print(magic_dict[i], lattice[i])
        cover_count += 1

print(cover_count / wrongs)



['朋友'] {'下课': 0, '歌星': 0, '个性': 0, '行期': 0, '星级': 0, '朋友': 0, '卸去': 0, '法国': 0, '返国': 0, '古玩': 0, '国王': 0}
['对不起'] {'对不起': 0, '不济': 0, '就近': 0, '最新': 0, '不讳': 0, '飞去': 0, '回去': 0}
['跳舞'] {'您们': 0, '还好': 0, '好话': 0, '跳舞': 0}
['早餐'] {'早餐': 0, '以后': 0, '后台': 0, '他处': 0, '去向': 0}
['走路'] {'走路': 0, '坐落': 0, '落差': 0, '路基': 0, '不多时': 0, '不独': 0, '多时': 0, '多石': 0, '多事': 0, '多士': 0, '时分': 0, '失分': 0, '闻到': 0}
['小鸡'] {'她们': 0, '它们': 0, '这个': 0, '三个': 0, '一根': 0, '爱迪': 0, '爱得': 0, '爱德': 0, '电音': 0, '电扇': 0, '下机': 0, '小家': 0, '小气': 0, '小鸡': 0, '小节': 0, '进一': 0, '机翼': 0, '机体': 0, '各人': 0, '何人': 0, '人居': 0, '人气': 0, '起火': 0, '灾星': 0}
['因为'] {'因为': 0, '应力': 0, '大学': 0, '地学': 0, '觉得': 0, '一同': 0, '异同': 0, '以东': 0, '以高': 0, '同伴': 0, '伴同': 0, '国学': 0}
['因为'] {'美眉': 0, '因为': 0, '应力': 0, '力学': 0, '学舍': 0, '设有': 0, '冷机': 0, '于今': 0, '以及': 0, '惊天': 0, '今夫': 0, '家室': 0, '教师': 0, '教士': 0, '市里': 0, '壁面': 0, '认得': 0, '乐得': 0, '热地': 0, '了不得': 0, '得标': 0, '不大': 0, '不时': 0, '不了': 0}
['的'] {'姜文': 0, '该文': 0, '艾文': 0

In [86]:
#get where bert fail

proto_preds = read_csv( "/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/" + "bert_MaskedLM_v2_std_mask.epoch20.bs32" + "/generated_predictions.txt")

for i in range(len(source[:1100])):
    proto_preds[i] = "".join(proto_preds[i].split())[:len(source[i])] 

def get_cor(source, target):
    return [ (j, target[j]) for j in range(len(target)) if target[j] != source[j]]

truth_cor_host, bert_cor_host, proto_cor_host = [], [], [] 

for i in range(len(source)):
    truth_cor, bert_cor, proto_cor = get_cor(source[i], target[i]), get_cor(source[i], raw_bert_preds[i]), get_cor(source[i], proto_preds[i])

    truth_cor_host.append(truth_cor)
    bert_cor_host.append(bert_cor)
    proto_cor_host.append(proto_cor)

bert_fail = []
plan = 0

two_wrong = 0
def isequal(truth_cor, now_cor):
    #cor: [(index, char), ...]
    #if equal
    if len(truth_cor) != len(now_cor):
        return False

    for i in range(len(truth_cor)):
        if truth_cor[i] != now_cor[i]:
            return False
    return True

#print(len())

for i in range(len(raw_bert_preds)):
    line = ["  "] * len(source[i]) + ["\n"]

    if not isequal(truth_cor_host[i], bert_cor_host[i]):
        if not bert_cor_host[i]:
            pass
        else:
            for cor in truth_cor_host[i]:
                line[cor[0]] = cor[1]
        line_back = "".join(line)
        line = "".join(line).split()
        if line:
            print(line)
        for l in line:
            if len(l) > 1:
                print(source[i])
                print(line_back)
                print(l, lattice[i])
                two_wrong += 1
                break
        

        bert_fail.append(line)


['记得']
看电影时候,我都觉得这个电影很有意思,可是现在我把什么事都不济的。
                                                            记得  

记得 {'看点': 0, '电影室': 0, '电影史': 0, '电音': 0, '电扇': 0, '一时': 0, '影视': 0, '影射': 0, '影史': 0, '侍候': 0, '我队': 0, '得知': 0, '一个': 0, '这儿': 0, '这根': 0, '有意识': 0, '有益': 0, '意识': 0, '可视': 0, '实现': 0, '现存': 0, '在握': 0, '怎么': 0, '公事': 0, '美事': 0, '不及': 0, '不计': 0, '不齐': 0, '不仅': 0}
['见']
['告', '糊涂']
我以前想要高诉你,可是我忘了。我真户秃。
          告                      糊涂  

糊涂 {'前线': 0, '前项': 0, '前嫌': 0, '险要': 0, '显要': 0, '女高': 0, '告诉': 0, '高速': 0, '可视': 0, '真空': 0, '真货': 0}
['啦']
['因', '怎']
['见']
['弟弟']
下星期是我的第第生日,他和我的父母想请他的两个朋友来。
            弟弟                                      

弟弟 {'下行': 0, '行期': 0, '星级': 0, '真是': 0, '期数': 0, '期市': 0, '圣日': 0, '生时': 0, '太和': 0, '泰和': 0, '由来': 0, '发来': 0}
['她', '去']
['女', '她']
['女']
['馆']
['书吗']
你可以告诉我那家书店有中文数马?
                          书吗  

书吗 {'妮可': 0, '课以': 0, '何以': 0, '可喜': 0, '可疑': 0, '以高': 0, '告示': 0, '你家': 0, '哪家': 0, '教书': 0, '家数': 0, '家当': 0, '家属': 0, '有种': 0, 

In [79]:
print(plan)
print(len(bert_fail), bert_fail)
print(two_wrong)


0
213 [[], [], ['记得'], ['见'], ['告', '糊涂'], [], [], [], [], ['啦'], [], ['因', '怎'], ['见'], ['弟弟'], [], [], [], [], [], [], ['她', '去'], [], [], ['女', '她'], ['女'], [], [], [], [], [], [], ['馆'], ['书吗'], [], [], ['名'], [], [], ['这样'], [], [], ['饱'], [], [], [], [], ['妈妈'], ['休', '吃饭好吗'], [], [], ['纪', '做', '忆'], ['儿'], ['候', '然'], ['议'], [], [], [], ['简讯'], ['卧房'], ['卧房'], ['诉'], [], ['交'], [], [], [], ['科'], ['因', '得'], [], [], [], ['提'], [], ['像'], [], [], ['再'], [], [], [], [], ['关'], ['征'], [], [], ['什'], [], [], [], ['半', '半'], [], [], ['需'], [], [], ['租'], ['轻'], ['只'], ['只', '牠', '只'], [], [], [], ['按'], [], ['消息'], ['儿地', '恢'], [], [], ['为'], [], [], [], [], [], ['这种', '整'], [], [], ['留', '游学'], [], [], [], [], ['个'], ['都', '都', '失'], ['地', '心'], [], ['心情好', '会'], ['地'], ['地'], [], [], ['做这'], [], ['候'], [], [], [], [], ['起', '赢'], [], [], ['著'], ['行'], [], [], [], ['家长', '时'], ['占'], [], ['个'], ['唠叨'], ['想'], ['的'], [], ['地'], [], [], ['拜托'], ['积', '堆', '到'], [], ['年轻'], ['品质'], ['

In [6]:
def compute_metrics_visuial(sources, preds, labels):
        """
        """
        tp, fp, fn = 0, 0, 0

        sent_p, sent_n = 0, 0

        for i in range(len(sources)):
            
            source, pred, label = sources[i], preds[i], labels[i]
            
            source = source[:len(label)]
            pred = pred[:len(label)]

            if not (pred == source):
                sent_p += 1
                if (pred == label):
                    tp += 1

            if (label != source):
                sent_n += 1

        precision = tp / sent_p
        recall = tp / sent_n
        F1_score = 2 * precision * recall / (precision + recall + 1e-10)

        print("precision: ", precision, "recall: ", recall, "F1_score: ", F1_score)

def short_show(name, ids_list, source, truth_cor_host, bert_cor_host, proto_cor_host):
    tmp = ""

    for i in ids_list:
        first = source[i] + "\n"
        second = ["  "] * len(source[i]) + ["\n"]
        for cor in truth_cor_host[i]:
            second[cor[0]] = cor[1]

        third = ["  "] * len(source[i]) + ["\n"]
        for cor in bert_cor_host[i]:
            third[cor[0]] = cor[1]

        fourth = ["  "] * len(source[i]) + ["\n"]
        for cor in proto_cor_host[i]:
            fourth[cor[0]] = cor[1]
        
        tmp += "".join(first) + "".join(second) + "".join(third) + "".join(fourth) + "\n"

    write_to("tmp_analysis_" + name + ".txt", tmp)

    return

def _path(name):
    return "/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/" + name + "/generated_predictions.txt"

def _analysis(name, path=None):
    if not path:
        proto_preds = read_csv(_path(name=name))
    else:
        proto_preds = read_csv(path)
    print(proto_preds[0], source[0])
    for i in range(len(source)):
        proto_preds[i] = "".join(proto_preds[i].split())[:len(source[i])] 

    need_cor = []
    no_need_cor = []

    #for i in range(len(source)):
    #    print(source[i])
    #    print(target[i])
    #    print(proto_preds[i])

    def get_cor(source, target):
        return [ (j, target[j]) for j in range(len(target)) if target[j] != source[j] and target not in "，。！？【】（）《》“‘"]

    truth_cor_host, bert_cor_host, proto_cor_host = [], [], [] 
    
    fn = 0

    print(len(source))

    for i in range(len(source)):
        truth_cor, bert_cor, proto_cor = get_cor(source[i], target[i]), get_cor(source[i], raw_bert_preds[i]), get_cor(source[i], proto_preds[i])

        truth_cor_host.append(truth_cor)
        bert_cor_host.append(bert_cor)
        proto_cor_host.append(proto_cor)

        #print(truth_cor, proto_cor)
        for cor in proto_cor:
            if cor not in truth_cor:
                fn += 1
        #if i == 5:
        #    break 
    print(fn)
    
    #形式化: 
    # sentnece
    #     en 
    joint = ""
    for i in range(len(source)):
        first = source[i] + "\n"
        second = ["  "] * len(source[i]) + ["\n"]
        if truth_cor_host[i]:
            for cor in truth_cor_host[i]:
                second[cor[0]] = cor[1]

        third = ["  "] * len(source[i]) + ["\n"]
        if bert_cor_host[i]:
            for cor in bert_cor_host[i]:
                third[cor[0]] = cor[1]

        fourth = ["  "] * len(source[i]) + ["\n"]
        if proto_cor_host[i]:
            for cor in proto_cor_host[i]:
                fourth[cor[0]] = cor[1]
        joint += "".join(first) + "".join(second) + "".join(third) + "".join(fourth) + "\n"

        write_to("tmp_analysis_" + name + ".txt", joint)

    #样本条数 proto比bert好的
    proto_better_than_bert = 0
    proto_worse_than_bert = 0

    def isequal(truth_cor, now_cor):
        #cor: [(index, char), ...]
        #if equal
        if len(truth_cor) != len(now_cor):
            return False

        for i in range(len(truth_cor)):
            if truth_cor[i] != now_cor[i]:
                return False
        return True

    better_list = []
    worse_list = []

    failed = []

    for i in range(len(truth_cor_host)):
        if not isequal(proto_cor_host[i], truth_cor_host[i]):
            failed.apepnd(i)
        if isequal(proto_cor_host[i], truth_cor_host[i]) and (not isequal(bert_cor_host[i], truth_cor_host[i])):
            proto_better_than_bert += 1
            better_list.append(i)
            #print(source[i])
        elif (not isequal(proto_cor_host[i], truth_cor_host[i])) and (isequal(bert_cor_host[i], truth_cor_host[i])):
            proto_worse_than_bert += 1
            worse_list.append(i)
        #break

    #check if lexcions perform
    """
    yjzheng_result = ['小鸡', '记得', '糊涂', '订位', '怎么', '面见', '弟弟', '炒饭', '汉字', '那店', '八点钟', '这样', '迟到', '很饱', '迟到', '迟到', '安静', '一点', '妈妈', '吃饭', '时候', '希望', '简讯', '那里', '诉取', '不半工', '哪些', '上半', '上半', '轻松', '消息', '与此同时', '这种', '权利', '优家', '失业', '心情', '减少', '著迷', '家长', '唠叨', '影响', '睡觉', '拜托', '存到', '年轻', '威胁', '我们', '回覆', '墙壁', '他们', '这是', '现在', '很忙', '不怎么', '试试看', '录影机', '哪个', '哪个', '不是', '再加', '该不该', '看著', '看著', '有装', '教书', '记录']
    print(len(yjzheng_result))

    yjzheng_ids = []
    hit = 0
    work = 0
    for i in range(len(source)):
        for j in yjzheng_result:
            if re.search(j, target[i]) and (not re.search(j, source[i])) and (not re.search(j, raw_bert_preds[i])):
                print(j, source[i], lattice[i])
                if j in lattice[i]:
                    hit += 1
                    if (re.search(j, proto_preds[i])):
                        work += 1
                else:
                    print("*"*3, j, source[i], "*"*3) 

    print(len(yjzheng_ids))
    print(yjzheng_ids)
    print("hit: ", hit)
    print("work: ", work)
    """
    print("better: ", proto_better_than_bert)
    print("worse: ", proto_worse_than_bert)

    short_show("failed_" + name, failed, source ,truth_cor_host, bert_cor_host, proto_cor_host)

    short_show("better_" + name, better_list, source ,truth_cor_host, bert_cor_host, proto_cor_host)

    short_show("worse_" + name, worse_list, source, truth_cor_host, bert_cor_host, proto_cor_host)
    
    print("BERT BaseLine: ")
    compute_metrics_visuial(source, raw_bert_preds, target)
    print("Proto :")
    compute_metrics_visuial(source, proto_preds, target)


In [5]:
_analysis(name="", path="/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/new_preds.txt")

你好！我是张爱文。
 你好！我是张爱文。
1100
(12, '得')
(17, '累')
(15, '再')
(11, '”')
(5, '纸')
(12, '得')
(17, '又')
(4, '进')
(0, '那')
(21, '候')
(12, '①')
(13, '着')
(14, '急')
(15, '那')
(16, '样')
(17, '子')
(18, '她')
(19, '知')
(20, '道')
(21, '①')
(22, '需')
(23, '要')
(24, '帮')
(25, '忙')
(26, '，')
(27, '所')
(28, '以')
(29, '她')
(30, '停')
(31, '下')
(32, '来')
(33, '向')
(34, '①')
(35, '问')
(36, '问')
(37, '。')
(14, '①')
(15, '讲')
(16, '话')
(17, '，')
(18, '然')
(19, '后')
(20, '帮')
(21, '①')
(22, '知')
(23, '道')
(24, '到')
(25, '他')
(26, '想')
(27, '去')
(28, '的')
(29, '地')
(30, '方')
(31, '怎')
(32, '么')
(33, '走')
(34, '。')
(0, '①')
(1, '很')
(2, '感')
(3, '谢')
(4, '那')
(5, '位')
(6, '女')
(7, '生')
(8, '。')
(4, '诉')
(5, '候')
(11, '心')
(7, '①')
(8, '②')
(9, '，')
(10, '这')
(11, '里')
(12, '有')
(13, '很')
(14, '有')
(15, '名')
(16, '的')
(17, '花')
(18, '地')
(19, '方')
(20, '。')
(11, '公')
(17, '得')
(0, '她')
(0, '她')
(20, '有')
(14, '得')
(8, '是')
(4, '复')
(13, '①')
(14, '唱')
(15, '歌')
(16, '，')
(17, '过')
(18, '了')
(19, '很')
(20, '开')
(21, 

AttributeError: 'list' object has no attribute 'apepnd'

In [12]:
_analysis("bert_MaskedLM_v2_std_mask_expand_d3.epoch10.bs32")

BERT BaseLine: 
precision:  0.7040816326530612 recall:  0.7638376383763837 F1_score:  0.7327433627819413
Proto :
precision:  0.6987522281639929 recall:  0.7232472324723247 F1_score:  0.7107887578829252


In [13]:
_analysis("bert_MaskedLM_v2_std_mask_dropout02_test.epoch10.bs32")

BERT BaseLine: 
precision:  0.7040816326530612 recall:  0.7638376383763837 F1_score:  0.7327433627819413
Proto :
precision:  0.7053571428571429 recall:  0.7287822878228782 F1_score:  0.7168784028538248


In [27]:
_analysis(name="bl_repeat14", path="bl_bert_repeat14.txt")

相对地、每位产龄妇女的生育婴儿个数却持续下滑。这表示全球出现适合年龄生育的妇女不想生育的现象。
 相对的、每位产龄妇女的生育婴儿个数却特续下滑。这表示全球出现适合年龄生育的妇女不想生育的现象。
BERT BaseLine: 
precision:  0.0 recall:  0.0 F1_score:  0.0
Proto :
precision:  0.7220394736842105 recall:  0.8442307692307692 F1_score:  0.7783687942765454


In [9]:
_analysis(name="bl_repeat", path="bl_bert_repeat.txt")

67
小鸡 他们看了一个很可爱的电影,一个小机一个人去火星。 {'她们': 0, '它们': 0, '这个': 0, '三个': 0, '一根': 0, '爱迪': 0, '爱得': 0, '爱德': 0, '电音': 0, '电扇': 0, '下机': 0, '小家': 0, '小气': 0, '小鸡': 0, '小节': 0, '进一': 0, '机翼': 0, '机体': 0, '各人': 0, '何人': 0, '人居': 0, '人气': 0, '起火': 0, '灾星': 0}
记得 看电影时候,我都觉得这个电影很有意思,可是现在我把什么事都不济的。 {'看点': 0, '电影室': 0, '电影史': 0, '电音': 0, '电扇': 0, '一时': 0, '影视': 0, '影射': 0, '影史': 0, '侍候': 0, '我队': 0, '得知': 0, '一个': 0, '这儿': 0, '这根': 0, '有意识': 0, '有益': 0, '意识': 0, '可视': 0, '实现': 0, '现存': 0, '在握': 0, '怎么': 0, '公事': 0, '美事': 0, '不及': 0, '不计': 0, '不齐': 0, '不仅': 0}
*** 记得 看电影时候,我都觉得这个电影很有意思,可是现在我把什么事都不济的。 ***
糊涂 我以前想要高诉你,可是我忘了。我真户秃。 {'前线': 0, '前项': 0, '前嫌': 0, '险要': 0, '显要': 0, '女高': 0, '告诉': 0, '高速': 0, '可视': 0, '真空': 0, '真货': 0}
*** 糊涂 我以前想要高诉你,可是我忘了。我真户秃。 ***
订位 所以我在"义大利面方子"已经定位了 {'一大': 0, '医大': 0, '意大利': 0, '奇利': 0, '达利': 0, '得利': 0, '戴利': 0, '大理': 0, '大力': 0, '里面': 0, '房子': 0, '一经': 0, '定为': 0, '为了': 0, '未了': 0}
*** 订位 所以我在"义大利面方子"已经定位了 ***
怎么 他们见面的时候,台生很高兴,应为外面真么那么热。 {'她们': 0, '它们': 0, '面见': 0, '建面': 

In [83]:
_analysis("bert_MaskedLM_v2_std_mask.epoch20.bs32")

67
小鸡 他们看了一个很可爱的电影,一个小机一个人去火星。 {'她们': 0, '它们': 0, '这个': 0, '三个': 0, '一根': 0, '爱迪': 0, '爱得': 0, '爱德': 0, '电音': 0, '电扇': 0, '下机': 0, '小家': 0, '小气': 0, '小鸡': 0, '小节': 0, '进一': 0, '机翼': 0, '机体': 0, '各人': 0, '何人': 0, '人居': 0, '人气': 0, '起火': 0, '灾星': 0}
记得 看电影时候,我都觉得这个电影很有意思,可是现在我把什么事都不济的。 {'看点': 0, '电影室': 0, '电影史': 0, '电音': 0, '电扇': 0, '一时': 0, '影视': 0, '影射': 0, '影史': 0, '侍候': 0, '我队': 0, '得知': 0, '一个': 0, '这儿': 0, '这根': 0, '有意识': 0, '有益': 0, '意识': 0, '可视': 0, '实现': 0, '现存': 0, '在握': 0, '怎么': 0, '公事': 0, '美事': 0, '不及': 0, '不计': 0, '不齐': 0, '不仅': 0}
*** 记得 看电影时候,我都觉得这个电影很有意思,可是现在我把什么事都不济的。 ***
糊涂 我以前想要高诉你,可是我忘了。我真户秃。 {'前线': 0, '前项': 0, '前嫌': 0, '险要': 0, '显要': 0, '女高': 0, '告诉': 0, '高速': 0, '可视': 0, '真空': 0, '真货': 0}
*** 糊涂 我以前想要高诉你,可是我忘了。我真户秃。 ***
订位 所以我在"义大利面方子"已经定位了 {'一大': 0, '医大': 0, '意大利': 0, '奇利': 0, '达利': 0, '得利': 0, '戴利': 0, '大理': 0, '大力': 0, '里面': 0, '房子': 0, '一经': 0, '定为': 0, '为了': 0, '未了': 0}
*** 订位 所以我在"义大利面方子"已经定位了 ***
怎么 他们见面的时候,台生很高兴,应为外面真么那么热。 {'她们': 0, '它们': 0, '面见': 0, '建面': 

In [15]:
_analysis("bert_MaskedLM_v2_std_mask_dropout_test.epoch10.bs32")

BERT BaseLine: 
precision:  0.7040816326530612 recall:  0.7638376383763837 F1_score:  0.7327433627819413
Proto :
precision:  0.7054673721340388 recall:  0.7380073800738007 F1_score:  0.7213706040979064


In [16]:
_analysis("bert_MaskedLM_v2_std_mask_hidden_dropout.epoch10.bs32")

BERT BaseLine: 
precision:  0.7040816326530612 recall:  0.7638376383763837 F1_score:  0.7327433627819413
Proto :
precision:  0.7034358047016275 recall:  0.7177121771217713 F1_score:  0.7105022830550279


In [19]:
_analysis("bert_MaskedLM_v2_std_mask_expand_d3.epoch10.bs32")

BERT BaseLine: 
precision:  0.7040816326530612 recall:  0.7638376383763837 F1_score:  0.7327433627819413
Proto :
precision:  0.6987522281639929 recall:  0.7232472324723247 F1_score:  0.7107887578829252
